In [14]:
import duckdb

In [25]:
con = duckdb.connect(database='dados_duckdb.db', read_only=False)

In [37]:
# Ensure the connection object `con` is defined and the previous cells are executed
df = con.execute("""
                SELECT *
                 FROM(
                 SELECT *, ROW_NUMBER() OVER (PARTITION BY Number ORDER BY data_ingestao DESC) AS row_num
                 FROM bronze_z002025                     
                 WHERE data_ingestao >= '2025-05-15'
                 ) WHERE row_num = 1
                 """).fetchdf()
df.head(5)

,Number,Material,Quantity,Price,arquivo,data_ingestao,row_num
0,101,Steel,100,50,z250535_2.csv,2025-05-15 09:27:22.982973,1
1,202,Aluminum,200,30,z250535_2.csv,2025-05-15 09:27:22.982973,1
2,303,Plastic,300,20,z250535_2.csv,2025-05-15 09:27:22.982973,1
3,404,Wood,400,90,z250535_2.csv,2025-05-15 09:27:22.982973,1


In [49]:
df_final = (df.drop(columns=['arquivo', 'data_ingestao', 'row_num']))
df_final = df_final.rename(columns= {"Number": "id"})
df_final = df_final.rename(columns= {'Material': "Materiais"})
df_final = df_final.rename(columns= {'Quantity': "Quantidade"})
df_final = df_final.rename(columns= {'Price': "Preço"})
df_final.head(12)

,id,Materiais,Quantidade,Preço
0,101,Steel,100,50
1,202,Aluminum,200,30
2,303,Plastic,300,20
3,404,Wood,400,90


In [52]:
df_final.dtypes

id            object
Materiais     object
Quantidade    object
Preço         object
dtype: object

In [ ]:
df2 = df_final
df2 = df2.astype(
    {
    'id': 'int',
    'Materiais': 'string',
    'Quantidade': 'string',
    'Preço': 'float32'
}
)
#df_final.head(12)
df2.dtypes

id                     int64
Materiais     string[python]
Quantidade    string[python]
Preço                float32
dtype: object

In [64]:
con.execute("""
                CREATE TABLE IF NOT EXISTS produtos(
            id BIGINT,
            Materiais STRING,
            Quantidade STRING,
            Preço FLOAT
            )
            """)

In [65]:
df2.head(12)

,id,Materiais,Quantidade,Preço
0,101,Steel,100,50.0
1,202,Aluminum,200,30.0
2,303,Plastic,300,20.0
3,404,Wood,400,90.0


In [70]:
con.execute(" INSERT INTO produtos SELECT * FROM df2") 


In [71]:
df_result = con.execute(" select * from produtos").fetchdf()
df_result.head(12)

,id,Materiais,Quantidade,Preço
0,101,Steel,100,50.0
1,202,Aluminum,200,30.0
2,303,Plastic,300,20.0
3,404,Wood,400,90.0


In [72]:
con.close()